In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import keyboard
import numpy as np

# Setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
drawing_spec = mp.solutions.drawing_utils.DrawingSpec(thickness=1, circle_radius=1)

control_enabled = False
screen_width, screen_height = pyautogui.size()

def get_eye_center(landmarks, eye_indices, frame_width, frame_height):
    x = int(np.mean([landmarks[i].x for i in eye_indices]) * frame_width)
    y = int(np.mean([landmarks[i].y for i in eye_indices]) * frame_height)
    return x, y

cap = cv2.VideoCapture(0)

LEFT_EYE = [33, 133, 160, 159, 158, 144, 153, 154, 155]
RIGHT_EYE = [362, 263, 387, 386, 385, 373, 380, 374, 381]

print("Press 't' to toggle control. Press 'q' to quit.")

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    if keyboard.is_pressed('q'):
        print("Exiting.")
        break

    if keyboard.is_pressed('t'):
        control_enabled = not control_enabled
        print(f"Control Enabled: {control_enabled}")
        keyboard.wait('t')  # Prevent rapid toggling

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    frame_height, frame_width = frame.shape[:2]

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        left_x, left_y = get_eye_center(landmarks, LEFT_EYE, frame_width, frame_height)
        right_x, right_y = get_eye_center(landmarks, RIGHT_EYE, frame_width, frame_height)

        gaze_x = int((left_x + right_x) / 2)
        gaze_y = int((left_y + right_y) / 2)

        # Map webcam coordinates to screen coordinates
        screen_x = np.interp(gaze_x, [0, frame_width], [0, screen_width])
        screen_y = np.interp(gaze_y, [0, frame_height], [0, screen_height])

        if control_enabled:
            pyautogui.moveTo(screen_x, screen_y, duration=0.05)

        # Debug display
        cv2.circle(frame, (gaze_x, gaze_y), 5, (0, 255, 0), -1)
        cv2.putText(frame, f"Mouse: {int(screen_x)}, {int(screen_y)}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow('Gaze Mouse Control', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
